In [1]:
import requests
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import Select

In [2]:
# selenium을 백그라운드로 실행하기 위해 옵션 설정
options = webdriver.ChromeOptions()
# options.add_argument('headless')
options.add_argument('window-size=1920x1080')

In [3]:
# 우리가 데이터를 가져올 웹 페이지입니다.
my_url = 'https://www.op.gg/summoner/userName=bbo+bbo'

# selenium이 제어할 chrome을 실행합니다.
driver = webdriver.Chrome('/Users/HEESU/downloads/chromedriver')

<ipython-input-3-a79b8961ce17>:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/Users/HEESU/downloads/chromedriver')


In [ ]:
# 데이터를 가져올 페이지로 이동합니다.
driver.get(my_url)

In [ ]:
driver.find_element_by_id('right_gametype_flexranked').click()
time.sleep(1)

In [ ]:
# '더 보기' 버튼이 활성화되어있으면(리스트의 끝까지 도달하지 않았으면) 계속 클릭하여 이전 게임을 모두 로딩합니다.
while True:
    try:
        driver.find_element_by_css_selector('.GameMoreButton').click()
        # 게임 로딩, html 코드 변경까지 여유 시간을 1초 가집니다.
        time.sleep(1)
        
    # 에러가 나면(페이지에서 '더 보기' 버튼이 없을 경우) while문을 탈출합니다.
    except Exception as e:
        pass
        break

In [ ]:
# 게임이 모두 로딩된 페이지의 html 코드를 faker_total_html로 저장합니다.
my_total_html = driver.page_source

# selenium이 제어하는 크롬을 종료합니다.
driver.quit()

In [ ]:
# html 형식에 맞춰 파싱(parsing; 추후 이용하기 쉽도록 쪼개기)합니다.
my_total_soup = BeautifulSoup(my_total_html, 'html.parser')

In [ ]:
# 전체 html 코드 중 우리가 원하는 selector를 만족하는 것만 가져오기
my_total_games_html = my_total_soup.select('div.GameItemList div.GameItemWrap')

# 그렇게 가져온 html 코드의 element 개수 == 끝까지 로딩된 모든 게임 수 출력
total_game_len = len(my_total_games_html)
print(total_game_len)

In [ ]:
# 각 게임에 대해 웹 페이지에 기재된 스탯을 찾아서(selector 사용) 결과 리스트에 append하기
my_mvp = 0
my_ace = 0
my_win = 0
my_lose = 0
for i in range(total_game_len):
    if list(my_total_games_html[i].select_one('div.GameResult').stripped_strings)[0] == "승리":
        my_win += 1
    elif list(my_total_games_html[i].select_one('div.GameResult').stripped_strings)[0] == "패배":
        my_lose += 1
        
    #ace나 mvp가 있는 판    
    if my_total_games_html[i].select_one('div.Badge') != None:
        if list(my_total_games_html[i].select_one('div.Badge').stripped_strings)[0] == "ACE":
            my_ace += 1
        elif list(my_total_games_html[i].select_one('div.Badge').stripped_strings)[0] == "MVP":
            my_mvp += 1
    

In [ ]:
mvp = [my_mvp, my_ace]
win_lose = [my_win, my_lose]

In [ ]:
my_df = pd.DataFrame([mvp, win_lose], index = ['MVP or ACE', 'total'], columns = ['win', 'lose'])

In [ ]:
my_df